In [49]:
import pandas as pd
import my_utils as mu

In [50]:
df = mu.get_rawdata("GOLD")

get summonerId....


100%|██████████| 4/4 [00:03<00:00,  1.30it/s]


get puuId.....


100%|██████████| 12/12 [00:08<00:00,  1.43it/s]


get match_id....


100%|██████████| 12/12 [00:09<00:00,  1.28it/s]


get matches & timeline....


100%|██████████| 36/36 [01:05<00:00,  1.83s/it]


In [117]:
def lane_processing(tower_lane,tower_team,lane,team):
    if(tower_lane[0]== lane[0]) and (tower_team == team):
        return 1
    else:
        return 0

In [118]:
def assi_calc(y):
    try:
        return ','.join(list(map(lambda x: str(x),y['assistingParticipantIds'])))
    except:
        return ''

In [130]:
df_creates = []
for i in range(len(df)):
    try:
        match = df.iloc[i].matches['info']['participants']
        for j in range(len(match)):
            match_id = df.iloc[i].match_id
            game_duration = df.iloc[i].matches['info']['gameDuration']
            game_version = df.iloc[i].matches['info']['gameVersion']
            try:
                riotIdGameName = match[j]['riotIdGameName']
            except:
                riotIdGameName = ''
            try:
                riotIdTagline = match[j]['riotIdTagline']
            except:
                riotIdTagline = ''
            summoner_level = match[j]['summonerLevel']
            participants_id = match[j]['participantId']
            champion_name = match[j]['championName']
            champ_experience = match[j]['champExperience']
            team_position = match[j]['teamPosition']
            team_id = match[j]['teamId']
            win = match[j]['win']
            kills = match[j]['kills']
            deaths = match[j]['deaths']
            assists = match[j]['assists']
            total_damage_dealt = match[j]['totalDamageDealtToChampions']
            total_damage_taken = match[j]['totalDamageTaken']

            df_list = list(map(lambda x: x['events'],df.iloc[i].timelines['info']['frames']))
            event_list = [element for array in df_list for element in array]
            tower_log = list(filter(lambda x:x['type']=='BUILDING_KILL', event_list))
            try:
                ft_tower_lane = tower_log[0]['laneType']
                ft_tower_team = tower_log[0]['teamId']
            except:
                ft_tower_lane = 'n'
                ft_tower_team = 'n'
            ft = lane_processing(ft_tower_lane,ft_tower_team,team_position,team_id)

            tower_tmp = list(map(lambda x:(x['laneType'],x['teamId'],x['timestamp']),tower_log))
            try:
                laneTower = list(filter(lambda x:(x[0][0]==team_position[0]) & (x[1]==team_id), tower_tmp))[0]
                lane_flag = 1
                laneTowerTime = laneTower[-1]
            except:
                lane_flag = 0
                laneTowerTime = 0
                
            blue_ban = list(map(lambda x: str(x['championId']), df.iloc[i].matches['info']['teams'][0]['bans']))
            red_ban = list(map(lambda x: str(x['championId']), df.iloc[i].matches['info']['teams'][1]['bans']))
            ban_list = 'l'.join(list(set(blue_ban + red_ban)))

            kill_log = list(map(lambda x: list(filter(lambda z : z['type']=='CHAMPION_KILL',x['events'])) ,df.iloc[i].timelines['info']['frames']))
            kill_log = [element for array in kill_log for element in array]
            k = 'l'.join(list(map(lambda x: str(x['killerId']) ,kill_log)))
            v = 'l'.join(list(map(lambda x: str(x['victimId']), kill_log)))
            a = 'l'.join(list(map(lambda x: assi_calc(x), kill_log)))
            try:
                g15 = df.iloc[i].timelines['info']['frames'][15]['participantFrames'][(str(j+1))]['totalGold']
            except:
                g15 =0
        
            df_creates.append([match_id,game_duration,game_version,riotIdGameName,riotIdTagline,summoner_level,participants_id,
                              champion_name,champ_experience,team_position,team_id,win,kills,deaths,assists,total_damage_dealt,total_damage_taken,
                              ft,lane_flag,laneTowerTime,ban_list,k,v,a,g15])
    except:
        continue
columns =['matchId', 'gameDuration', 'gameVersion', 'riotIdGameName','riotIdTagline', 
           'summonerLevel', 'participantId', 'championName', 'champExperience', 
           'teamPosition', 'teamId', 'win', 'kills', 'deaths', 'assists', 'totalDamageDealtToChampions', 'totalDamageTaken',
         'firstDT','laneTower','LaneTowerTime','bans','killerId','victimId','assistId','G15']
df2 = pd.DataFrame(df_creates , columns = columns)

In [131]:
df2

,matchId,gameDuration,gameVersion,riotIdGameName,riotIdTagline,summonerLevel,participantId,championName,champExperience,teamPosition,...,totalDamageDealtToChampions,totalDamageTaken,firstDT,laneTower,LaneTowerTime,bans,killerId,victimId,assistId,G15
0,KR_6848397207,1632,13.24.547.9214,FridayYa,KR1,604,1,Renekton,13047,TOP,...,9865,33166,0,1,1074955,134l233l48l78l53l41l60l25l777l360,7l7l9l10l2l6l9l7l1l9l6l2l7l6l9l3l9l4l6l2l7l9l9...,3l1l5l4l6l2l4l5l6l4l1l6l3l2l5l10l4l9l1l6l3l4l5...,"8,9,10l6l10l7,9l1ll10ll2l10l7ll8,10ll10l4,5l10...",5381
1,KR_6848397207,1632,13.24.547.9214,Bee Wind,KR1,332,2,LeeSin,11690,JUNGLE,...,15689,32185,0,0,0,134l233l48l78l53l41l60l25l777l360,7l7l9l10l2l6l9l7l1l9l6l2l7l6l9l3l9l4l6l2l7l9l9...,3l1l5l4l6l2l4l5l6l4l1l6l3l2l5l10l4l9l1l6l3l4l5...,"8,9,10l6l10l7,9l1ll10ll2l10l7ll8,10ll10l4,5l10...",5901
2,KR_6848397207,1632,13.24.547.9214,Fiftd,KR1,273,3,Yasuo,12382,MIDDLE,...,21638,22529,1,1,649694,134l233l48l78l53l41l60l25l777l360,7l7l9l10l2l6l9l7l1l9l6l2l7l6l9l3l9l4l6l2l7l9l9...,3l1l5l4l6l2l4l5l6l4l1l6l3l2l5l10l4l9l1l6l3l4l5...,"8,9,10l6l10l7,9l1ll10ll2l10l7ll8,10ll10l4,5l10...",5693
3,KR_6848397207,1632,13.24.547.9214,비정상적인이름,KR1,303,4,Ziggs,10003,BOTTOM,...,18630,15790,0,1,1116808,134l233l48l78l53l41l60l25l777l360,7l7l9l10l2l6l9l7l1l9l6l2l7l6l9l3l9l4l6l2l7l9l9...,3l1l5l4l6l2l4l5l6l4l1l6l3l2l5l10l4l9l1l6l3l4l5...,"8,9,10l6l10l7,9l1ll10ll2l10l7ll8,10ll10l4,5l10...",4596
4,KR_6848397207,1632,13.24.547.9214,푸른수첩,KR1,206,5,Xerath,7950,UTILITY,...,19677,15498,0,0,0,134l233l48l78l53l41l60l25l777l360,7l7l9l10l2l6l9l7l1l9l6l2l7l6l9l3l9l4l6l2l7l9l9...,3l1l5l4l6l2l4l5l6l4l1l6l3l2l5l10l4l9l1l6l3l4l5...,"8,9,10l6l10l7,9l1ll10ll2l10l7ll8,10ll10l4,5l10...",5357
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
305,KR_6675650749,1561,13.16.525.6443,,,495,6,Swain,10679,TOP,...,17310,31083,0,0,0,526l950l876l111l60l7l-1l777l164,7l7l3l1l6l3l10l7l5l2l3l8l7l7l10l7l2l1l4l6l4l3l...,5l2l6l8l3l8l4l5l9l7l10l1l2l3l5l3l7l8l10l1l9l6l...,"9,10l9,10llll1l7,9l9,10l4l3l2l6l6l6l8,9l6l1ll5...",5461
306,KR_6675650749,1561,13.16.525.6443,,,207,7,Khazix,12570,JUNGLE,...,12508,27579,0,0,0,526l950l876l111l60l7l-1l777l164,7l7l3l1l6l3l10l7l5l2l3l8l7l7l10l7l2l1l4l6l4l3l...,5l2l6l8l3l8l4l5l9l7l10l1l2l3l5l3l7l8l10l1l9l6l...,"9,10l9,10llll1l7,9l9,10l4l3l2l6l6l6l8,9l6l1ll5...",7040
307,KR_6675650749,1561,13.16.525.6443,,,101,8,Zed,10570,MIDDLE,...,21125,21292,0,1,1270182,526l950l876l111l60l7l-1l777l164,7l7l3l1l6l3l10l7l5l2l3l8l7l7l10l7l2l1l4l6l4l3l...,5l2l6l8l3l8l4l5l9l7l10l1l2l3l5l3l7l8l10l1l9l6l...,"9,10l9,10llll1l7,9l9,10l4l3l2l6l6l6l8,9l6l1ll5...",4863
308,KR_6675650749,1561,13.16.525.6443,,,607,9,Ashe,8325,BOTTOM,...,10366,16162,0,1,1041491,526l950l876l111l60l7l-1l777l164,7l7l3l1l6l3l10l7l5l2l3l8l7l7l10l7l2l1l4l6l4l3l...,5l2l6l8l3l8l4l5l9l7l10l1l2l3l5l3l7l8l10l1l9l6l...,"9,10l9,10llll1l7,9l9,10l4l3l2l6l6l6l8,9l6l1ll5...",4448


In [132]:
tower_tmp = list(map(lambda x:(x['laneType'],x['teamId'],x['timestamp']),tower_log))

In [133]:
tower_tmp

[('TOP_LANE', 100, 922104),
 ('BOT_LANE', 200, 1041491),
 ('BOT_LANE', 100, 1125003),
 ('MID_LANE', 100, 1182707),
 ('MID_LANE', 200, 1270182),
 ('MID_LANE', 200, 1419030),
 ('MID_LANE', 200, 1430758),
 ('MID_LANE', 200, 1435569),
 ('BOT_LANE', 200, 1438403),
 ('BOT_LANE', 200, 1452866),
 ('MID_LANE', 200, 1548018),
 ('MID_LANE', 200, 1551244)]

In [134]:
laneTower = list(filter(lambda x:(x[0][0]==team_position[0]) & (x[1]==team_id), tower_tmp))[0]

IndexError: list index out of range

In [135]:
laneTower

('BOT_LANE', 200, 1041491)

In [136]:
tower_tmp

[('TOP_LANE', 100, 922104),
 ('BOT_LANE', 200, 1041491),
 ('BOT_LANE', 100, 1125003),
 ('MID_LANE', 100, 1182707),
 ('MID_LANE', 200, 1270182),
 ('MID_LANE', 200, 1419030),
 ('MID_LANE', 200, 1430758),
 ('MID_LANE', 200, 1435569),
 ('BOT_LANE', 200, 1438403),
 ('BOT_LANE', 200, 1452866),
 ('MID_LANE', 200, 1548018),
 ('MID_LANE', 200, 1551244)]

In [200]:
blue_team = df2[df2.teamId==100]
red_team = df2[df2.teamId==200]

In [201]:
tmp_b = red_team[['matchId','gameVersion','teamPosition','championName','G15']]

In [202]:
tmp_b.columns = 'matchId', 'gameVersion', 'teamPosition', 'enemy_champ', 'enemy_G15'

In [203]:
tmp_r = blue_team[['matchId','gameVersion','teamPosition','championName','G15']]

In [204]:
tmp_r.columns = 'matchId', 'gameVersion', 'teamPosition', 'enemy_champ', 'enemy_G15'

In [205]:
tmp_r

,matchId,gameVersion,teamPosition,enemy_champ,enemy_G15
0,KR_6848397207,13.24.547.9214,TOP,Renekton,5381
1,KR_6848397207,13.24.547.9214,JUNGLE,LeeSin,5901
2,KR_6848397207,13.24.547.9214,MIDDLE,Yasuo,5693
3,KR_6848397207,13.24.547.9214,BOTTOM,Ziggs,4596
4,KR_6848397207,13.24.547.9214,UTILITY,Xerath,5357
...,...,...,...,...,...
300,KR_6675650749,13.16.525.6443,MIDDLE,Nasus,5658
301,KR_6675650749,13.16.525.6443,JUNGLE,Vi,4606
302,KR_6675650749,13.16.525.6443,TOP,Syndra,5960
303,KR_6675650749,13.16.525.6443,BOTTOM,Ezreal,4371


In [206]:
blue_team = pd.merge(blue_team,tmp_b,how='inner',on= ['matchId','gameVersion','teamPosition'])

In [207]:
red_team = pd.merge(red_team,tmp_r,how='inner',on= ['matchId','gameVersion','teamPosition'])

In [208]:
red_team

,matchId,gameDuration,gameVersion,riotIdGameName,riotIdTagline,summonerLevel,participantId,championName,champExperience,teamPosition,...,firstDT,laneTower,LaneTowerTime,bans,killerId,victimId,assistId,G15,enemy_champ,enemy_G15
0,KR_6848397207,1632,13.24.547.9214,레고한가득삼킴,KR1,382,6,MissFortune,13958,TOP,...,0,1,1016792,134l233l48l78l53l41l60l25l777l360,7l7l9l10l2l6l9l7l1l9l6l2l7l6l9l3l9l4l6l2l7l9l9...,3l1l5l4l6l2l4l5l6l4l1l6l3l2l5l10l4l9l1l6l3l4l5...,"8,9,10l6l10l7,9l1ll10ll2l10l7ll8,10ll10l4,5l10...",5807,Renekton,5381
1,KR_6848397207,1632,13.24.547.9214,싹재명,2323,67,7,Belveth,13232,JUNGLE,...,0,0,0,134l233l48l78l53l41l60l25l777l360,7l7l9l10l2l6l9l7l1l9l6l2l7l6l9l3l9l4l6l2l7l9l9...,3l1l5l4l6l2l4l5l6l4l1l6l3l2l5l10l4l9l1l6l3l4l5...,"8,9,10l6l10l7,9l1ll10ll2l10l7ll8,10ll10l4,5l10...",7273,LeeSin,5901
2,KR_6848397207,1632,13.24.547.9214,보노를 보노,KR1,355,8,Vladimir,14819,MIDDLE,...,0,0,0,134l233l48l78l53l41l60l25l777l360,7l7l9l10l2l6l9l7l1l9l6l2l7l6l9l3l9l4l6l2l7l9l9...,3l1l5l4l6l2l4l5l6l4l1l6l3l2l5l10l4l9l1l6l3l4l5...,"8,9,10l6l10l7,9l1ll10ll2l10l7ll8,10ll10l4,5l10...",5109,Yasuo,5693
3,KR_6848397207,1632,13.24.547.9214,좀 섹시한 원딜,KR1,333,9,Kaisa,12597,BOTTOM,...,0,1,1546270,134l233l48l78l53l41l60l25l777l360,7l7l9l10l2l6l9l7l1l9l6l2l7l6l9l3l9l4l6l2l7l9l9...,3l1l5l4l6l2l4l5l6l4l1l6l3l2l5l10l4l9l1l6l3l4l5...,"8,9,10l6l10l7,9l1ll10ll2l10l7ll8,10ll10l4,5l10...",7135,Ziggs,4596
4,KR_6848397207,1632,13.24.547.9214,감자헌,KR1,182,10,Nautilus,9634,UTILITY,...,0,0,0,134l233l48l78l53l41l60l25l777l360,7l7l9l10l2l6l9l7l1l9l6l2l7l6l9l3l9l4l6l2l7l9l9...,3l1l5l4l6l2l4l5l6l4l1l6l3l2l5l10l4l9l1l6l3l4l5...,"8,9,10l6l10l7,9l1ll10ll2l10l7ll8,10ll10l4,5l10...",4311,Xerath,5357
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
150,KR_6675650749,1561,13.16.525.6443,,,495,6,Swain,10679,TOP,...,0,0,0,526l950l876l111l60l7l-1l777l164,7l7l3l1l6l3l10l7l5l2l3l8l7l7l10l7l2l1l4l6l4l3l...,5l2l6l8l3l8l4l5l9l7l10l1l2l3l5l3l7l8l10l1l9l6l...,"9,10l9,10llll1l7,9l9,10l4l3l2l6l6l6l8,9l6l1ll5...",5461,Syndra,5960
151,KR_6675650749,1561,13.16.525.6443,,,207,7,Khazix,12570,JUNGLE,...,0,0,0,526l950l876l111l60l7l-1l777l164,7l7l3l1l6l3l10l7l5l2l3l8l7l7l10l7l2l1l4l6l4l3l...,5l2l6l8l3l8l4l5l9l7l10l1l2l3l5l3l7l8l10l1l9l6l...,"9,10l9,10llll1l7,9l9,10l4l3l2l6l6l6l8,9l6l1ll5...",7040,Vi,4606
152,KR_6675650749,1561,13.16.525.6443,,,101,8,Zed,10570,MIDDLE,...,0,1,1270182,526l950l876l111l60l7l-1l777l164,7l7l3l1l6l3l10l7l5l2l3l8l7l7l10l7l2l1l4l6l4l3l...,5l2l6l8l3l8l4l5l9l7l10l1l2l3l5l3l7l8l10l1l9l6l...,"9,10l9,10llll1l7,9l9,10l4l3l2l6l6l6l8,9l6l1ll5...",4863,Nasus,5658
153,KR_6675650749,1561,13.16.525.6443,,,607,9,Ashe,8325,BOTTOM,...,0,1,1041491,526l950l876l111l60l7l-1l777l164,7l7l3l1l6l3l10l7l5l2l3l8l7l7l10l7l2l1l4l6l4l3l...,5l2l6l8l3l8l4l5l9l7l10l1l2l3l5l3l7l8l10l1l9l6l...,"9,10l9,10llll1l7,9l9,10l4l3l2l6l6l6l8,9l6l1ll5...",4448,Ezreal,4371


In [209]:
team_df = pd.concat([blue_team,red_team])

In [297]:
team_df

,matchId,gameDuration,gameVersion,riotIdGameName,riotIdTagline,summonerLevel,participantId,championName,champExperience,teamPosition,...,LaneTowerTime,bans,killerId,victimId,assistId,G15,enemy_champ,enemy_G15,lane_win,first_blood
0,KR_6848397207,1632,13.24.547.9214,FridayYa,KR1,604,1,Renekton,13047,TOP,...,1074955,134l233l48l78l53l41l60l25l777l360,7l7l9l10l2l6l9l7l1l9l6l2l7l6l9l3l9l4l6l2l7l9l9...,3l1l5l4l6l2l4l5l6l4l1l6l3l2l5l10l4l9l1l6l3l4l5...,"8,9,10l6l10l7,9l1ll10ll2l10l7ll8,10ll10l4,5l10...",5381,MissFortune,5807,0,none
1,KR_6848397207,1632,13.24.547.9214,Bee Wind,KR1,332,2,LeeSin,11690,JUNGLE,...,0,134l233l48l78l53l41l60l25l777l360,7l7l9l10l2l6l9l7l1l9l6l2l7l6l9l3l9l4l6l2l7l9l9...,3l1l5l4l6l2l4l5l6l4l1l6l3l2l5l10l4l9l1l6l3l4l5...,"8,9,10l6l10l7,9l1ll10ll2l10l7ll8,10ll10l4,5l10...",5901,Belveth,7273,0,none
2,KR_6848397207,1632,13.24.547.9214,Fiftd,KR1,273,3,Yasuo,12382,MIDDLE,...,649694,134l233l48l78l53l41l60l25l777l360,7l7l9l10l2l6l9l7l1l9l6l2l7l6l9l3l9l4l6l2l7l9l9...,3l1l5l4l6l2l4l5l6l4l1l6l3l2l5l10l4l9l1l6l3l4l5...,"8,9,10l6l10l7,9l1ll10ll2l10l7ll8,10ll10l4,5l10...",5693,Vladimir,5109,1,none
3,KR_6848397207,1632,13.24.547.9214,비정상적인이름,KR1,303,4,Ziggs,10003,BOTTOM,...,1116808,134l233l48l78l53l41l60l25l777l360,7l7l9l10l2l6l9l7l1l9l6l2l7l6l9l3l9l4l6l2l7l9l9...,3l1l5l4l6l2l4l5l6l4l1l6l3l2l5l10l4l9l1l6l3l4l5...,"8,9,10l6l10l7,9l1ll10ll2l10l7ll8,10ll10l4,5l10...",4596,Kaisa,7135,0,none
4,KR_6848397207,1632,13.24.547.9214,푸른수첩,KR1,206,5,Xerath,7950,UTILITY,...,0,134l233l48l78l53l41l60l25l777l360,7l7l9l10l2l6l9l7l1l9l6l2l7l6l9l3l9l4l6l2l7l9l9...,3l1l5l4l6l2l4l5l6l4l1l6l3l2l5l10l4l9l1l6l3l4l5...,"8,9,10l6l10l7,9l1ll10ll2l10l7ll8,10ll10l4,5l10...",5357,Nautilus,4311,1,none
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
150,KR_6675650749,1561,13.16.525.6443,,,495,6,Swain,10679,TOP,...,0,526l950l876l111l60l7l-1l777l164,7l7l3l1l6l3l10l7l5l2l3l8l7l7l10l7l2l1l4l6l4l3l...,5l2l6l8l3l8l4l5l9l7l10l1l2l3l5l3l7l8l10l1l9l6l...,"9,10l9,10llll1l7,9l9,10l4l3l2l6l6l6l8,9l6l1ll5...",5461,Syndra,5960,0,none
151,KR_6675650749,1561,13.16.525.6443,,,207,7,Khazix,12570,JUNGLE,...,0,526l950l876l111l60l7l-1l777l164,7l7l3l1l6l3l10l7l5l2l3l8l7l7l10l7l2l1l4l6l4l3l...,5l2l6l8l3l8l4l5l9l7l10l1l2l3l5l3l7l8l10l1l9l6l...,"9,10l9,10llll1l7,9l9,10l4l3l2l6l6l6l8,9l6l1ll5...",7040,Vi,4606,1,none
152,KR_6675650749,1561,13.16.525.6443,,,101,8,Zed,10570,MIDDLE,...,1270182,526l950l876l111l60l7l-1l777l164,7l7l3l1l6l3l10l7l5l2l3l8l7l7l10l7l2l1l4l6l4l3l...,5l2l6l8l3l8l4l5l9l7l10l1l2l3l5l3l7l8l10l1l9l6l...,"9,10l9,10llll1l7,9l9,10l4l3l2l6l6l6l8,9l6l1ll5...",4863,Nasus,5658,0,none
153,KR_6675650749,1561,13.16.525.6443,,,607,9,Ashe,8325,BOTTOM,...,1041491,526l950l876l111l60l7l-1l777l164,7l7l3l1l6l3l10l7l5l2l3l8l7l7l10l7l2l1l4l6l4l3l...,5l2l6l8l3l8l4l5l9l7l10l1l2l3l5l3l7l8l10l1l9l6l...,"9,10l9,10llll1l7,9l9,10l4l3l2l6l6l6l8,9l6l1ll5...",4448,Ezreal,4371,1,none


In [296]:
team_df['lane_win'] = team_df.apply(lambda x: 1 if x['G15']>x['enemy_G15'] else 0, axis=1)

In [301]:
team_df['first_blood'] = team_df.apply(lambda x : 1 if str(x.participantId) == x.killerId[0] else 0 , axis=1)

In [302]:
team_df

,matchId,gameDuration,gameVersion,riotIdGameName,riotIdTagline,summonerLevel,participantId,championName,champExperience,teamPosition,...,LaneTowerTime,bans,killerId,victimId,assistId,G15,enemy_champ,enemy_G15,lane_win,first_blood
0,KR_6848397207,1632,13.24.547.9214,FridayYa,KR1,604,1,Renekton,13047,TOP,...,1074955,134l233l48l78l53l41l60l25l777l360,7l7l9l10l2l6l9l7l1l9l6l2l7l6l9l3l9l4l6l2l7l9l9...,3l1l5l4l6l2l4l5l6l4l1l6l3l2l5l10l4l9l1l6l3l4l5...,"8,9,10l6l10l7,9l1ll10ll2l10l7ll8,10ll10l4,5l10...",5381,MissFortune,5807,0,0
1,KR_6848397207,1632,13.24.547.9214,Bee Wind,KR1,332,2,LeeSin,11690,JUNGLE,...,0,134l233l48l78l53l41l60l25l777l360,7l7l9l10l2l6l9l7l1l9l6l2l7l6l9l3l9l4l6l2l7l9l9...,3l1l5l4l6l2l4l5l6l4l1l6l3l2l5l10l4l9l1l6l3l4l5...,"8,9,10l6l10l7,9l1ll10ll2l10l7ll8,10ll10l4,5l10...",5901,Belveth,7273,0,0
2,KR_6848397207,1632,13.24.547.9214,Fiftd,KR1,273,3,Yasuo,12382,MIDDLE,...,649694,134l233l48l78l53l41l60l25l777l360,7l7l9l10l2l6l9l7l1l9l6l2l7l6l9l3l9l4l6l2l7l9l9...,3l1l5l4l6l2l4l5l6l4l1l6l3l2l5l10l4l9l1l6l3l4l5...,"8,9,10l6l10l7,9l1ll10ll2l10l7ll8,10ll10l4,5l10...",5693,Vladimir,5109,1,0
3,KR_6848397207,1632,13.24.547.9214,비정상적인이름,KR1,303,4,Ziggs,10003,BOTTOM,...,1116808,134l233l48l78l53l41l60l25l777l360,7l7l9l10l2l6l9l7l1l9l6l2l7l6l9l3l9l4l6l2l7l9l9...,3l1l5l4l6l2l4l5l6l4l1l6l3l2l5l10l4l9l1l6l3l4l5...,"8,9,10l6l10l7,9l1ll10ll2l10l7ll8,10ll10l4,5l10...",4596,Kaisa,7135,0,0
4,KR_6848397207,1632,13.24.547.9214,푸른수첩,KR1,206,5,Xerath,7950,UTILITY,...,0,134l233l48l78l53l41l60l25l777l360,7l7l9l10l2l6l9l7l1l9l6l2l7l6l9l3l9l4l6l2l7l9l9...,3l1l5l4l6l2l4l5l6l4l1l6l3l2l5l10l4l9l1l6l3l4l5...,"8,9,10l6l10l7,9l1ll10ll2l10l7ll8,10ll10l4,5l10...",5357,Nautilus,4311,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
150,KR_6675650749,1561,13.16.525.6443,,,495,6,Swain,10679,TOP,...,0,526l950l876l111l60l7l-1l777l164,7l7l3l1l6l3l10l7l5l2l3l8l7l7l10l7l2l1l4l6l4l3l...,5l2l6l8l3l8l4l5l9l7l10l1l2l3l5l3l7l8l10l1l9l6l...,"9,10l9,10llll1l7,9l9,10l4l3l2l6l6l6l8,9l6l1ll5...",5461,Syndra,5960,0,0
151,KR_6675650749,1561,13.16.525.6443,,,207,7,Khazix,12570,JUNGLE,...,0,526l950l876l111l60l7l-1l777l164,7l7l3l1l6l3l10l7l5l2l3l8l7l7l10l7l2l1l4l6l4l3l...,5l2l6l8l3l8l4l5l9l7l10l1l2l3l5l3l7l8l10l1l9l6l...,"9,10l9,10llll1l7,9l9,10l4l3l2l6l6l6l8,9l6l1ll5...",7040,Vi,4606,1,1
152,KR_6675650749,1561,13.16.525.6443,,,101,8,Zed,10570,MIDDLE,...,1270182,526l950l876l111l60l7l-1l777l164,7l7l3l1l6l3l10l7l5l2l3l8l7l7l10l7l2l1l4l6l4l3l...,5l2l6l8l3l8l4l5l9l7l10l1l2l3l5l3l7l8l10l1l9l6l...,"9,10l9,10llll1l7,9l9,10l4l3l2l6l6l6l8,9l6l1ll5...",4863,Nasus,5658,0,0
153,KR_6675650749,1561,13.16.525.6443,,,607,9,Ashe,8325,BOTTOM,...,1041491,526l950l876l111l60l7l-1l777l164,7l7l3l1l6l3l10l7l5l2l3l8l7l7l10l7l2l1l4l6l4l3l...,5l2l6l8l3l8l4l5l9l7l10l1l2l3l5l3l7l8l10l1l9l6l...,"9,10l9,10llll1l7,9l9,10l4l3l2l6l6l6l8,9l6l1ll5...",4448,Ezreal,4371,1,0


7

'7l7l9l10l2l6l9l7l1l9l6l2l7l6l9l3l9l4l6l2l7l9l9l2l9l5l5l3l2l7l3l5l9l2l0l2l9l3l3l8l10l2l2l6l8l9l6l8l7l8l5l9l2l9l7l7l9l2'